In [93]:
import jax
import jax.numpy as jnp
from jax import jit
import jax.random as rnd
import numpy as np
np.set_printoptions(threshold=1000, linewidth=200)
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker
import os
import functools

def load_data(str_):
    path_ = os.path.dirname(os.getcwd()) + '/sc_project/test_data/' # Path(__file__).resolve().parents[1]
    with open(path_+str_,'rb') as file_:
        # param = pickle.load(file_)
        param_ = jnp.load(file_,allow_pickle=True)
    return param_

file_ = 'forward_new_v1_225_31_07-0617.pkl' # 'move_loop_v2_data_27_07-160547.pkl' # 'move_loop_v2_data_27_07-161313.pkl'[7] # _'move_loop_v1_26_07-093804.pkl' # move_loop_v1__25_07-161050.pkl
(loss_arr,loss_sem,v_pred_arr,v_t_arr,pos_arr,dot_arr,opt_state,p_weights) = load_data(file_) # # [VMAPS,STEPS,N]x2,[VMAPS,STEPS,2]x3,[VMAPS,STEPS]x2
print('loss_arr.shape=',loss_arr.shape,'loss_sem.shape=',loss_sem.shape,'v_pred_arr.shape=',v_pred_arr.shape,'v_t_arr.shape=',v_t_arr.shape,'pos_arr.shape=',pos_arr.shape,'dot_arr.shape=',dot_arr.shape)

loss_arr.shape= (10000,) loss_sem.shape= (10000,) v_pred_arr.shape= (500, 6, 300) v_t_arr.shape= (500, 6, 300) pos_arr.shape= (500, 2, 7) dot_arr.shape= (500, 2, 7)


In [102]:
# full ani
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.animation as animation
import functools

def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def gen_vectors(MODULES, APERTURE):
    m = MODULES
    A = APERTURE
    x = np.linspace(-(A-A/m), (A-A/m), m)
    x_ = np.tile(x, (m, ))
    y_ = np.repeat(np.flip(x), m)
    return np.vstack([x_, y_])

def set_axis_properties(ax,lim,labels,title=''):
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_aspect('equal', 'box')  
    ax.set_title(title)
    ticks = np.linspace(-lim,lim,5)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

def get_aperture_points(agent_t,aperture,density=100):
    x_horizontal = np.linspace(agent_t[0] - aperture, agent_t[0] + aperture, density)
    y_top = np.full(density, agent_t[1] + aperture)
    y_bottom = np.full(density, agent_t[1] - aperture)

    y_vertical = np.linspace(agent_t[1] - aperture, agent_t[1] + aperture, density)
    x_left = np.full(density, agent_t[0] - aperture)
    x_right = np.full(density, agent_t[0] + aperture)

    x_tot = np.concatenate([x_left, x_right, x_horizontal, x_horizontal])
    y_tot = np.concatenate([y_vertical, y_vertical, y_bottom, y_top])
    return x_tot, y_tot

def animate(i,v_pred_arr,v_t_arr,dot_arr,pos_arr,dot,agent,neuron_locs,scatters_pred,scatters_t,aperture_points):
    # initialize
    dot.set_data(mod_(dot_arr[i, 0]), mod_(dot_arr[i, 1]))
    agent.set_data(mod_(pos_arr[i, 0]), mod_(pos_arr[i, 1]))
    # Update v_pred_arr
    scatters_pred.set_offsets(neuron_locs.T)
    scatters_pred.set_array(v_pred_arr[i,:,:].flatten())
    scatters_pred.set_sizes(17 * np.sum(v_pred_arr[i, :, :], axis=0))
    # Update v_t_arr
    scatters_t.set_offsets(neuron_locs.T)
    scatters_t.set_array(v_t_arr[i,:,:].flatten())
    scatters_t.set_sizes(17 * np.sum(v_t_arr[i, :, :], axis=0))
    x_aperture,y_aperture = get_aperture_points(pos_arr[i],APERTURE)
    aperture_points.set_offsets(np.c_[mod_(x_aperture), mod_(y_aperture)])
    return [dot,agent,scatters_pred,scatters_t,aperture_points]

k = 0
interval = 300
STEPS = v_pred_arr.shape[1]
MODULES = 15
NEURONS = 10
APERTURE = np.pi/2
ACT_SPACE = np.pi/5

neuron_locs = gen_vectors(NEURONS,ACT_SPACE)
v_pred_arr_ = np.clip(v_pred_arr[k,:,:],0,1).reshape((STEPS,3,NEURONS**2)) #.transpose(0, 2, 1)
v_t_arr_ = v_t_arr[k,:,:].reshape((STEPS,3,NEURONS**2)) #.transpose(0, 2, 1)
dot_arr_ = dot_arr[k,:,:-1].T
pos_arr_ = pos_arr[k,:,:-1].T

fig = plt.figure(figsize=(8, 15))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1])

ax_ani = plt.subplot(gs[0:2, 0:2],aspect='equal')
dot, = ax_ani.plot(mod_(dot_arr[0, 0]), mod_(dot_arr[0, 1]), 'rx')
agent, = ax_ani.plot(mod_(pos_arr[0, 0]), mod_(pos_arr[0, 1]), 'k+')
set_axis_properties(ax_ani,jnp.pi,["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"])

ax_pred = plt.subplot(gs[2, 0],aspect='equal')
scatters_pred = ax_pred.scatter(neuron_locs[0, :], neuron_locs[1, :], c=v_pred_arr_[0, :, :].T, s=17 * np.sum(v_pred_arr_[0, :, :], axis=0), marker='o')
set_axis_properties(ax_pred,ACT_SPACE,["$-\pi/5$", "$-\pi/10$", "0", "$\pi/10$", "$\pi/5$"],'v_pred')

ax_t = plt.subplot(gs[2, 1],aspect='equal')
scatters_t = ax_t.scatter(neuron_locs[0, :], neuron_locs[1, :], c=v_t_arr_[0, :, :].T, s=17 * np.sum(v_t_arr_[0, :, :], axis=0), marker='o')
set_axis_properties(ax_t,ACT_SPACE,["$-\pi/5$", "$-\pi/10$", "0", "$\pi/10$", "$\pi/5$"],'v_t')

aperture_points = ax_ani.scatter([], [], color='black', s=1)

partial_animate = functools.partial(
    animate,
    v_pred_arr=v_pred_arr_, 
    v_t_arr=v_t_arr_, 
    dot_arr=dot_arr_, 
    pos_arr=pos_arr_, 
    dot=dot, 
    agent=agent, 
    neuron_locs=neuron_locs, 
    scatters_pred=scatters_pred, 
    scatters_t=scatters_t,
    aperture_points=aperture_points,
)

ani = animation.FuncAnimation(
    fig=fig, 
    func=partial_animate,
    frames=STEPS, 
    blit=True, 
    interval=interval
)

plt.tight_layout()
plt.show()

TypeError: animate() missing 1 required positional argument: 'aperture_points'

Traceback (most recent call last):
  File "/Users/lukej/Documents/meta_rl_ego_sim/venv3/lib/python3.8/site-packages/matplotlib/cbook/__init__.py", line 309, in process
    func(*args, **kwargs)
  File "/Users/lukej/Documents/meta_rl_ego_sim/venv3/lib/python3.8/site-packages/matplotlib/animation.py", line 904, in _start
    self._init_draw()
  File "/Users/lukej/Documents/meta_rl_ego_sim/venv3/lib/python3.8/site-packages/matplotlib/animation.py", line 1748, in _init_draw
    self._draw_frame(frame_data)
  File "/Users/lukej/Documents/meta_rl_ego_sim/venv3/lib/python3.8/site-packages/matplotlib/animation.py", line 1767, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
TypeError: animate() missing 1 required positional argument: 'aperture_points'


In [99]:
# test ani
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec

# Create the main figure
# Keeping the width and height ratio in mind, using 6x8 ensures square subplots
fig = plt.figure(figsize=(6, 8))

# Create a GridSpec layout with 3 rows and 2 columns
gs = GridSpec(3, 2, figure=fig)

# Using the GridSpec layout, add three subplots
ax1 = fig.add_subplot(gs[0:2, :])  # Top square subplot spans first two rows and both columns
ax2 = fig.add_subplot(gs[2, 0])    # Bottom left square subplot
ax3 = fig.add_subplot(gs[2, 1])    # Bottom right square subplot

# Sample data
x = np.linspace(0, 2 * np.pi, 100)

# Initial plots
line1, = ax1.plot(x, np.sin(x))
line2, = ax2.plot(x, np.sin(2 * x))
line3, = ax3.plot(x, np.sin(3 * x))

def update(frame):
    """Update function for the animation"""
    line1.set_ydata(np.sin(x + frame * 0.1))
    line2.set_ydata(np.sin(2 * x + frame * 0.2))
    line3.set_ydata(np.sin(3 * x + frame * 0.3))
    return line1, line2, line3

# Create the animation
ani = FuncAnimation(fig, update, frames=np.arange(0, 100), blit=True)

plt.tight_layout()
plt.show()


/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_50180/837863753.py:34: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  dot.set_data(mod_(dot_arr[i, 0]), mod_(dot_arr[i, 1]))
/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_50180/837863753.py:35: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  agent.set_data(mod_(pos_arr[i, 0]), mod_(pos_arr[i, 1]))


In [98]:
# BACKUP DONT DELETE:
# 2
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.animation as animation
import functools

def mod_(x):
    return (x + np.pi) % (2 * np.pi) - np.pi

def gen_vectors(MODULES, APERTURE):
    m = MODULES
    A = APERTURE
    x = np.linspace(-(A-A/m), (A-A/m), m)
    x_ = np.tile(x, (m, ))
    y_ = np.repeat(np.flip(x), m)
    return np.vstack([x_, y_])

def set_axis_properties(ax,lim,labels,title=''):
    ax.set_xlim(-lim,lim)
    ax.set_ylim(-lim,lim)
    ax.set_aspect('equal', 'box')  
    ax.set_title(title)
    ticks = np.linspace(-lim,lim,5)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)

def animate(i, v_pred_arr, v_t_arr, dot_arr, pos_arr, dot, agent, neuron_locs, scatters_pred, scatters_t):
    # initialize
    dot.set_data(mod_(dot_arr[i, 0]), mod_(dot_arr[i, 1]))
    agent.set_data(mod_(pos_arr[i, 0]), mod_(pos_arr[i, 1]))

    # Update v_pred_arr
    scatters_pred.set_offsets(neuron_locs.T)
    scatters_pred.set_array(v_pred_arr[i,:,:].flatten())
    scatters_pred.set_sizes(17 * np.sum(v_pred_arr[i, :, :], axis=0))
    # Update v_t_arr
    scatters_t.set_offsets(neuron_locs.T)
    scatters_t.set_array(v_t_arr[i,:,:].flatten())
    scatters_t.set_sizes(17 * np.sum(v_t_arr[i, :, :], axis=0))

    return [dot, agent, scatters_pred, scatters_t]

k = 0
interval = 300
STEPS = v_pred_arr.shape[1]
MODULES = 15
NEURONS = 10
APERTURE = np.pi/5

neuron_locs = gen_vectors(NEURONS, APERTURE)
v_pred_arr_ = np.clip(v_pred_arr[k,:,:],0,1).reshape((STEPS,3,NEURONS**2)) #.transpose(0, 2, 1)
v_t_arr_ = v_t_arr[k,:,:].reshape((STEPS,3,NEURONS**2)) #.transpose(0, 2, 1)
dot_arr_ = dot_arr[k,:,:-1].T
pos_arr_ = pos_arr[k,:,:-1].T

fig = plt.figure(figsize=(10, 15))
gs = gridspec.GridSpec(3, 2, height_ratios=[1, 1, 1])

ax_ani = plt.subplot(gs[0:2, 0:2],aspect='equal')
dot, = ax_ani.plot(mod_(dot_arr[0, 0]), mod_(dot_arr[0, 1]), 'rx')
agent, = ax_ani.plot(mod_(pos_arr[0, 0]), mod_(pos_arr[0, 1]), 'k+')
set_axis_properties(ax_ani,jnp.pi,["$-\pi$", "$-\pi/2$", "0", "$\pi/2$", "$\pi$"])

ax_pred = plt.subplot(gs[2, 0],aspect='equal')
scatters_pred = ax_pred.scatter(neuron_locs[0, :], neuron_locs[1, :], c=v_pred_arr_[0, :, :].T, s=17 * np.sum(v_pred_arr_[0, :, :], axis=0), marker='o')
set_axis_properties(ax_pred,APERTURE,["$-\pi/5$", "$-\pi/10$", "0", "$\pi/10$", "$\pi/5$"],'v_pred')

ax_t = plt.subplot(gs[2, 1],aspect='equal')
scatters_t = ax_t.scatter(neuron_locs[0, :], neuron_locs[1, :], c=v_t_arr_[0, :, :].T, s=17 * np.sum(v_t_arr_[0, :, :], axis=0), marker='o')
set_axis_properties(ax_t,APERTURE,["$-\pi/5$", "$-\pi/10$", "0", "$\pi/10$", "$\pi/5$"],'v_t')

partial_animate = functools.partial(
    animate,
    v_pred_arr=v_pred_arr_, 
    v_t_arr=v_t_arr_, 
    dot_arr=dot_arr_, 
    pos_arr=pos_arr_, 
    dot=dot, 
    agent=agent, 
    neuron_locs=neuron_locs, 
    scatters_pred=scatters_pred, 
    scatters_t=scatters_t
)

ani = animation.FuncAnimation(
    fig=fig, 
    func=partial_animate,
    frames=STEPS, 
    blit=True, 
    interval=interval
)

plt.tight_layout()
plt.show()

/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_50180/837863753.py:34: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  dot.set_data(mod_(dot_arr[i, 0]), mod_(dot_arr[i, 1]))
/var/folders/hp/sdqbmwnd5m14lcp1k2qjnkx00000gn/T/ipykernel_50180/837863753.py:35: MatplotlibDeprecationWarning: Setting data with a non sequence type is deprecated since 3.7 and will be remove two minor releases later
  agent.set_data(mod_(pos_arr[i, 0]), mod_(pos_arr[i, 1]))
